In [14]:
import numpy as np
import random
import time

'''
这个代码实现了强化学习（Reinforcement Learning, RL）中的Q-learning 算法，在 4×4 迷宫（Gridworld） 环境中训练一个智能体（Agent）找到从 (0,0) 到 (3,3) 的最佳路径。
Q-learning 是强化学习（Reinforcement Learning, RL）中的一种无模型（Model-Free）学习算法，它可以在未知环境中通过**试错（Trial and Error）**不断优化决策，最终学到最优策略（Optimal Policy）。
'''

'''
 强化学习 (RL) 概述

强化学习是一种智能体与环境交互的学习方法，其中：

环境（Environment）：一个状态空间，定义了智能体可以在其中移动。
智能体（Agent）：在环境中学习如何行动的对象。
动作（Actions）：智能体在每个状态下可以选择的行为。
奖励（Reward）：智能体执行动作后获得的反馈，指导学习。
状态（State）：智能体在某一时刻所处的情况。
在这个代码中：

环境是一个 4x4 网格迷宫，智能体从 (0,0) 出发，到达 (3,3) 为终点。
智能体可以选择 "UP"、"DOWN"、"LEFT"、"RIGHT" 四个动作。
每移动一步奖励 -1，终点奖励 +100，智能体要学习最快到达终点的策略。
'''

# 定义环境大小
GRID_SIZE = 8
ACTIONS = ['UP', 'DOWN', 'LEFT', 'RIGHT']
LEARNING_RATE = 0.1   # 学习率
DISCOUNT_FACTOR = 0.9  # 折扣因子
EPSILON = 0.2          # 探索率（ε-greedy）

# 初始化 Q 表 (4x4x4) -> 4x4 的网格，每个格子有 4 个动作的 Q 值
Q_table = np.zeros((GRID_SIZE, GRID_SIZE, len(ACTIONS)))

# 奖励表
reward_table = -1 * np.ones((GRID_SIZE, GRID_SIZE))  # 默认奖励 -1
reward_table[GRID_SIZE - 1, GRID_SIZE - 1] = 100  # 终点奖励 +100

# 定义动作的偏移量
action_map = {
    'UP': (-1, 0),
    'DOWN': (1, 0),
    'LEFT': (0, -1),
    'RIGHT': (0, 1)
}

# 选择动作（ε-greedy）
def choose_action(state):
    x, y = state
    if random.uniform(0, 1) < EPSILON:
        return random.choice(ACTIONS)  # 随机探索
    else:
        return ACTIONS[np.argmax(Q_table[x, y])]  # 选择最高 Q 值的动作

# 执行动作，返回新状态和奖励
def step(state, action):
    x, y = state
    dx, dy = action_map[action]
    new_x, new_y = max(0, min(GRID_SIZE - 1, x + dx)), max(0, min(GRID_SIZE - 1, y + dy))  # 限制边界
    return (new_x, new_y), reward_table[new_x, new_y]

# Q-learning 训练
def train(episodes=500):
    for episode in range(episodes):
        state = (0, 0)  # 初始位置
        total_reward = 0

        while state != (GRID_SIZE - 1, GRID_SIZE - 1):  # 直到到达终点
            action = choose_action(state)  # 选择动作
            next_state, reward = step(state, action)  # 执行动作
            x, y = state
            action_index = ACTIONS.index(action)   #获得action对应的序号：    'UP': 0；'DOWN': 1；'LEFT': 2；'RIGHT': 3.

            # Q 值更新公式
            Q_table[x, y, action_index] = (1 - LEARNING_RATE) * Q_table[x, y, action_index] + \
                                          LEARNING_RATE * (reward + DISCOUNT_FACTOR * np.max(Q_table[next_state])) #np.max(Q_table[next_state])：输出next_state四个方向中Q_table最大的那个值

            state = next_state  # 更新状态
            total_reward += reward
        
        if episode % 50 == 0:
            print(f"Episode {episode}, Total Reward: {total_reward}")

# 运行训练
train()

# 测试最佳路径
def test():
    state = (0, 0)
    path = [state]
    while state != (GRID_SIZE - 1, GRID_SIZE - 1):
        action = ACTIONS[np.argmax(Q_table[state])] #每一步都选之前学习出来的Q_table最大的那一个方向
        state, _ = step(state, action)
        path.append(state)
    
    print("\nOptimal Path Found by Q-learning:")
    print(path)

# 测试训练结果
test()


Episode 0, Total Reward: -302.0
Episode 50, Total Reward: 53.0
Episode 100, Total Reward: 57.0
Episode 150, Total Reward: 80.0
Episode 200, Total Reward: 81.0
Episode 250, Total Reward: 84.0
Episode 300, Total Reward: 87.0
Episode 350, Total Reward: 83.0
Episode 400, Total Reward: 77.0
Episode 450, Total Reward: 85.0

Optimal Path Found by Q-learning:
[(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (2, 3), (3, 3), (4, 3), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7)]
